# Client 동기화

In [ ]:
import kfp
import requests

USERNAME = "user@example.com" # keycloak 사용시 불필요
PASSWORD = "12341234" # keycloak 사용시 불필요
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://127.0.0.1:8080" # istio-ingressgateway pod ip:port ex) http://istio-ingressgateway.istio-system/ 

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}", # 크롬창 kubeflow dashboard 에서 관리자도구 -> storage -> cookies -> authservice_session 토큰 복붙
)
list_pipelines = client.list_pipelines()

print(list_pipelines.total_size)

#for i in range(list_pipelines.total_size):

## Pipeline Upload

In [426]:
pipeline_package_path='/Users/gimgihun/file_transfer/kfp-v2/iris_csv.yaml' # pipeline yaml 파일 경로 수정해야함.
pipeline_name='iris-v1' # pipeline 이름 
client.upload_pipeline(pipeline_package_path=pipeline_package_path,pipeline_name=pipeline_name)

{'created_at': datetime.datetime(2023, 1, 16, 14, 10, 24, tzinfo=tzutc()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2023, 1, 16, 14, 10, 24, tzinfo=tzutc()),
                     'description': None,
                     'id': 'e29621de-9669-4c6a-891c-92c47f199343',
                     'name': 'iris-v1',
                     'package_url': None,
                     'parameters': [{'name': 'pipeline-root', 'value': None},
                                    {'name': 'pipeline-name',
                                     'value': 'pipeline/iris-v1'}],
                     'resource_references': [{'key': {'id': 'e29621de-9669-4c6a-891c-92c47f199343',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': 'e29621de-9669-4c6a-891c-92c47f

# Experiment 생성

In [427]:
experiment = client.create_experiment(name=pipeline_name,description=pipeline_name,namespace=NAMESPACE)

In [428]:
experiment_info = client.get_experiment(experiment_name=pipeline_name,namespace=NAMESPACE)
experiment_id = experiment_info.id
experiment_id


'af5c0c9e-216a-433d-99b2-4f00714866d9'

## Run 생성

In [429]:
client.run_pipeline(experiment_id=experiment.id,job_name=pipeline_name,pipeline_package_path=pipeline_package_path)

{'created_at': datetime.datetime(2023, 1, 16, 14, 10, 26, tzinfo=tzutc()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()),
 'id': '19a064c6-d443-4dcb-9396-b2f8941e36c9',
 'metrics': None,
 'name': 'iris-v1',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'pipeline_name': None,
                   'runtime_config': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": "iris-", '
                                        '"annotations": '
                                        '{"pipelines.kubeflow.org/kfp_sdk_version": '
                                        '"1.8.18", '
                                        '"pipelines.kubef

In [422]:
pipeline_id = client.get_pipeline_id(pipeline_name)
client.delete_pipeline(pipeline_id=pipeline_id)

{}

In [423]:
experiment_info = client.get_experiment(experiment_name=pipeline_name,namespace=NAMESPACE)
experiment_id = experiment_info.id

In [424]:
client.delete_experiment(experiment_id=experiment_id)

{}

In [425]:
for i in range(client.list_runs().total_size):
    run_id = client.list_runs().runs[i].id
    delete_run_id = run_id
    print(delete_run_id)

client.runs.delete_run(id=delete_run_id) # id값을 인자로 넣어주고 runs 삭제

8372d4b4-5b5c-43ee-b9fa-20074e7fa10a
af92870d-35a1-47a1-8bed-20c88bb416d6
8aca9d3b-62fa-452c-9fad-6539c6cb0e6f
c601a2d6-634b-470e-b7d5-0524f7cd6457


{}

In [ ]:
client.create_run_from_pipeline_package